In [1]:
import SplinesNLines
import numpy as np
import torch

m = 2 
B = 5
O = 2300
T = np.linspace(0,10, B+m+2)
Lp = 3
np.random.seed(201763)
mu = np.array([1.0, 3.0, 4.0, 1.5, np.pi, -1, 2.3, 0.7])
L = np.random.normal(size = B + Lp).reshape(-1,1)
S = 100*L@L.T + 2*np.eye(B + Lp)

print(S)

O0 = np.random.multivariate_normal(mean = mu, cov = S, size = O)

K_vec = np.random.randint(50, 100, O)


X_a = []
f_a = []
T_a = []

mu_v = []
for o in range(O):
    Tau = np.sort(np.random.uniform(size = K_vec[o]))
    To = np.random.uniform(0,10, size = K_vec[o])
    X_B = SplinesNLines.get_basis_mat_B_Spline_opt(To, B, m, T)
    X_L = SplinesNLines.eval_Line_Profiles_opt(To, loc_vals= np.array([1.5, 3.5, 7.5]), width_vals= np.array([1, 2.0, 0.5]))
    X = np.hstack((X_B, X_L))
    f = np.random.multivariate_normal(mean = X@O0[o, :], cov = np.diag(1/Tau), size = 1 )
    X_a.append(X)
    T_a.append(np.diag(Tau))
    f_a.append(f.T)
    mu_v.append(np.mean(f))


f_a2 = [f/np.abs(np.mean(mu_v)) for f in f_a]
A = SplinesNLines.get_mat_comp(X_a, f_a2, T_a)
l = A['XtTX'][5]



L = SplinesNLines.EM_alg(A['XtXf'], A['XtTX'], n_max = 10000, tol = 1e-10)

mu_h = L['mu_h']
S_h = L['S_h']

A2 = SplinesNLines.get_mat_comp(X_a, f_a, T_a)
L2 = SplinesNLines.EM_alg(A2['XtXf'], A2['XtTX'], n_max = 10000, tol = 1e-10)

mu_h2 = L['mu_h']
S_h2 = L['S_h']







[[ 36.22848187 -54.69945994  29.78026711  -5.39249239  13.45617719
   41.7983902   -8.71970938  -8.45931056]
 [-54.69945994  89.4134859  -47.59090789   8.61757242 -21.5038934
  -66.79669226  13.93469321  13.51855805]
 [ 29.78026711 -47.59090789  27.91012691  -4.69170279  11.70745909
   36.36641642  -7.5865262   -7.35996791]
 [ -5.39249239   8.61757242  -4.69170279   2.84955489  -2.11994015
   -6.58508613   1.37373801   1.33271373]
 [ 13.45617719 -21.5038934   11.70745909  -2.11994015   7.29000104
   16.43212068  -3.42796257  -3.32559248]
 [ 41.7983902  -66.79669226  36.36641642  -6.58508613  16.43212068
   53.04244558 -10.64814433 -10.33015618]
 [ -8.71970938  13.93469321  -7.5865262    1.37373801  -3.42796257
  -10.64814433   4.22134689   2.15501026]
 [ -8.45931056  13.51855805  -7.35996791   1.33271373  -3.32559248
  -10.33015618   2.15501026   4.09065466]]
Iteration:   0
Iteration:   1000
Iteration:   2000
Iteration:   3000
Iteration:   4000
Iteration:   5000
Iteration:   6000
Itera

In [5]:
mu_h2 = L2['mu_h']
S_h2 = L2['S_h']

#S= torch.from_numpy(S)
#mu = torch.from_numpy(mu)

print("Absolute error for mu in % ")
print(100*torch.abs(np.abs(np.mean(mu_v))*mu_h - mu.reshape(-1,1) )/torch.abs(mu.reshape(-1,1)))

print("Absolute error for S entries in %")
print(100*torch.abs(S - S_h*(np.mean(mu_v)**2))/torch.abs(S))




print("Absolute error for mu2 in % ")
print(100*torch.abs(mu_h2 - mu.reshape(-1,1) )/torch.abs(mu.reshape(-1,1)))
print("Absolute error for Sh2 entries in %")

print(100*torch.abs(S - S_h2)/torch.abs(S))



Absolute error for mu in % 
tensor([[ 37.9565],
        [ 30.4445],
        [ 13.0429],
        [ 13.3996],
        [  5.0364],
        [ 83.9292],
        [164.2704],
        [ 12.3015]], dtype=torch.float64)
Absolute error for S entries in %
tensor([[ 4.1121,  3.9184,  0.3573, 11.7247,  0.3732,  3.6484, 49.3714,  1.3831],
        [ 3.9184,  7.4642,  4.1917, 15.5704,  2.1231,  0.4556, 49.6593,  5.3337],
        [ 0.3573,  4.1917,  4.9544, 28.6802,  1.7616,  2.7424, 57.0898,  0.6246],
        [11.7247, 15.5704, 28.6802, 53.4591, 21.9722, 10.7169, 20.4997,  0.3536],
        [ 0.3732,  2.1231,  1.7616, 21.9722, 18.0429,  2.7865, 42.2182, 20.3095],
        [ 3.6484,  0.4556,  2.7424, 10.7169,  2.7865,  2.4973, 58.3609,  1.9024],
        [49.3714, 49.6593, 57.0898, 20.4997, 42.2182, 58.3609, 62.4087, 71.8738],
        [ 1.3831,  5.3337,  0.6246,  0.3536, 20.3095,  1.9024, 71.8738, 35.5039]],
       dtype=torch.float64)
Absolute error for mu2 in % 
tensor([[ 5.0319],
        [ 4.9624],
    